In [ ]:
!pip install vk_api
import requests
import vk_api
import random
from vk_api.longpoll import VkLongPoll, VkEventType
import joblib
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator




vk_session = vk_api.VkApi(token='Token')



def softmax(x):
    proba = np.exp(-x)
    return proba/sum(proba)

class NeighborSampler(BaseEstimator):
    def __init__(self, k=5, temp=1.0):
        self.k = k
        self.temp = temp
    
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
    
    def predict(self, X, random_state=None):
        distances, indeces = self.tree_.query(X, return_distance=True, k=self.k)
        result = []
        for distance, index in zip(distances, indeces):
            result.append(np.random.choice(index, p=softmax(distance*self.temp)))
        return self.y_[result]

clf2 = joblib.load(r'C:\Users\usov_\VkBot-master\VkBot\speak_bot_vk.joblib.pkl')

longpoll = VkLongPoll(vk_session)
vk = vk_session.get_api()
for event in longpoll.listen():
    if event.type == VkEventType.MESSAGE_NEW and event.to_me and event.text:

   #Слушаем longpoll, если пришло сообщение то:
        vk.messages.send( #Отправляем сообщение
                    user_id=event.user_id,
                    message=clf2.predict([event.text])[0],
                    random_id = random.random()
                )
                
        


In [16]:
import sklearn
sklearn.__version__

'0.23.2'